# Building SLP and MLP for Breast Cancer Data

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.decomposition import PCA as sklearnPCA

# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings("ignore")

---

# 4. Explore Dataset
## 4-1) Import dataset

In [ ]:
wbcd = pd.read_csv("../input/breast/breast-cancer-wisconsin.csv")
wbcd.head()

In [ ]:
wbcd = wbcd.replace('?', 0)

In [ ]:
print("This WBCD dataset is consisted of",wbcd.shape)

## 4-3) Summary the Diagnosis

In [ ]:
sns.countplot(wbcd['class'],label="Count")

## 4-4) Correlation Plot of 30 features
except **id, diagnosis** columns => wbcd.iloc[:,2:]

In [ ]:
corr = wbcd.iloc[:,:].corr()
colormap = sns.diverging_palette(420, 10, as_cmap = True)
plt.figure(figsize=(8,8))
sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 8},
            cmap = colormap, linewidths=0.1, linecolor='white')
plt.title('Correlation of Breast Cancer Data Features', y=1.05, size=15)

---

# 5. Preparing Data for machine learning
## 5-1) Divide "Breast Cancer data" into Train(70%) / Test data(30%)
Divide the data into two(train/test) to see the predictive power of the model.


In [ ]:
train,test = train_test_split(wbcd, test_size=0.3, random_state=42)
print("Training Data :",train.shape)
print("Testing Data :",test.shape)

In [ ]:
train_data = train
test_data = test


print("Training Data :",train_data.shape)
print("Testing Data :",test_data.shape)

In [ ]:
train.iloc[:,0:-1].head()

In [ ]:
train_x = train_data.iloc[:,0:-1]
train_x = MinMaxScaler().fit_transform(train_x)
print("Training Data :", train_x.shape)

# Testing Data
test_x = test_data.iloc[:,0:-1]
test_x = MinMaxScaler().fit_transform(test_x)
print("Testing Data :", test_x.shape)


In [ ]:
train_y = train_data.iloc[:,-1:]
train_y[train_y== 2] = 0
train_y[train_y== 4 ] = 1
print("Training Data :", train_y.shape)

# Testing Data
test_y = test_data.iloc[:,-1:]
test_y[test_y== 2] = 0
test_y[test_y==4 ] = 1
print("Testing Data :", test_y.shape)

---

# 6. Make ANN-SLP Model
## 6-1) Make "Placeholder" for dinamic variable allocation
Placeholder is one of the function in tensorflow.
It is a space to put and change values while the program is running.
* for X, a place must have 30 columns, since wbcd data has 30 features.
* for Y, a place must have 1 columns, since the results has 1 outcome.
* If you see the row "None", it means it has no size limits. (You can write -1 instead of "None")

In [ ]:
X = tf.placeholder(tf.float32, [None,9])
Y = tf.placeholder(tf.float32, [None, 1])

## 6-2) Make Weight, Bias value with randomly
* W(weight) : why **[30,1]**?  16 for 16 features, 1 for 1 Outcome(results).
* P(weight): why **[10,1]**? 10 for 10 PCA features, 1 for 1 Outcome(results).
* b(bias) : why **[1]**?  outcome has 1 layers.

In [ ]:
# weight
W = tf.Variable(tf.random_normal([9,1], seed=0), name='weight')

# bias
b = tf.Variable(tf.random_normal([1], seed=0), name='bias')

## 6-3) Make Output Results
 * **Output = Weight * Input + Bias**
 * tf.matmul() : for array multiply

In [ ]:
logits = tf.matmul(X,W) + b

In [ ]:
hypothesis = tf.nn.sigmoid(logits)

cost_i = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=Y)
cost = tf.reduce_mean(cost_i)
# cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))

In [ ]:
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

## 6-6) Compare : original vs. prediction

In [ ]:
prediction = tf.cast(hypothesis > 0.5, dtype=tf.float32)
correct_prediction = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))

## 6-7) Activate Model

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(10001):
        sess.run(train, feed_dict={X: train_x, Y: train_y})
        if step % 1000 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X: train_x, Y: train_y})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))
            
    train_acc = sess.run(accuracy, feed_dict={X: train_x, Y: train_y})
    test_acc,test_predict,test_correct = sess.run([accuracy,prediction,correct_prediction], feed_dict={X: test_x, Y: test_y})
    print("Model Prediction =", train_acc)
    print("Test Prediction =", test_acc)

---

# 7. ANN Model Summary & Compare
## 7-1) ANN - SLP Model
* train_x, test_x : normalization data
* 9 features
* train_y, test_y

In [ ]:
def ann_slp():
    print("===========Data Summary===========")
    print("Training Data :", train_x.shape)
    print("Testing Data :", test_x.shape)

    X = tf.placeholder(tf.float32, [None,9])
    Y = tf.placeholder(tf.float32, [None, 1])

    W = tf.Variable(tf.random_normal([9,1], seed=0), name='weight')
    b = tf.Variable(tf.random_normal([1], seed=0), name='bias')

    logits = tf.matmul(X,W) + b
    hypothesis = tf.nn.sigmoid(logits)
    
    cost_i = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=Y)
    cost = tf.reduce_mean(cost_i)

    train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

    prediction = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    correct_prediction = tf.equal(prediction, Y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))

    
    loss_ans= []
    acc_ans = []
    print("\n============Processing============")
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for step in range(10001):
            sess.run(train, feed_dict={X: train_x, Y: train_y})
            if step % 1000 == 0:
                loss, acc = sess.run([cost, accuracy], feed_dict={X: train_x, Y: train_y})
                loss_ans.append(loss)
                acc_ans.append(acc)
                print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))

        train_acc = sess.run(accuracy, feed_dict={X: train_x, Y: train_y})
        test_acc,test_predict,test_correct = sess.run([accuracy,prediction,correct_prediction], feed_dict={X: test_x, Y: test_y})
        
        print("\n============Results============")
        print("Model Prediction =", train_acc)
        print("Test Prediction =", test_acc)
        
        return train_acc,test_acc, loss_ans, acc_ans
    
ann_slp_train_acc, ann_slp_test_acc, loss_ans, acc_ans = ann_slp()

In [ ]:
#import seaborn as sn
sns.pointplot([i for i in range(len(loss_ans))], loss_ans)


## 7-3) ANN - MLP Model
* train_x, test_x : normalization data
* 9 features
* train_y, test_y

In [ ]:
def ann_mlp():
    print("===========Data Summary===========")
    print("Training Data :", train_x.shape)
    print("Testing Data :", test_x.shape)

    X = tf.placeholder(tf.float32, [None,9])
    Y = tf.placeholder(tf.float32, [None, 1])

    # input
    W1 = tf.Variable(tf.random_normal([9,18], seed=0), name='weight1')
    b1 = tf.Variable(tf.random_normal([18], seed=0), name='bias1')
    layer1 = tf.nn.sigmoid(tf.matmul(X,W1) + b1)

    # hidden1
    W2 = tf.Variable(tf.random_normal([18,18], seed=0), name='weight2')
    b2 = tf.Variable(tf.random_normal([18], seed=0), name='bias2')
    layer2 = tf.nn.sigmoid(tf.matmul(layer1,W2) + b2)

    # hidden2
    W3 = tf.Variable(tf.random_normal([18,27], seed=0), name='weight3')
    b3 = tf.Variable(tf.random_normal([27], seed=0), name='bias3')
    layer3 = tf.nn.sigmoid(tf.matmul(layer2,W3) + b3)

    # output
    W4 = tf.Variable(tf.random_normal([27,1], seed=0), name='weight4')
    b4 = tf.Variable(tf.random_normal([1], seed=0), name='bias4')
    logits = tf.matmul(layer3,W4) + b4
    hypothesis = tf.nn.sigmoid(logits)

    cost_i = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=Y)
    cost = tf.reduce_mean(cost_i)

    train = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost)

    prediction = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    correct_prediction = tf.equal(prediction, Y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))

    loss_ans = []
    print("\n============Processing============")
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for step in range(10001):
            sess.run(train, feed_dict={X: train_x, Y: train_y})
            if step % 1000 == 0:
                loss, acc = sess.run([cost, accuracy], feed_dict={X: train_x, Y: train_y})
                
                print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))
            loss_ans.append(loss)

        train_acc = sess.run(accuracy, feed_dict={X: train_x, Y: train_y})
        test_acc,test_predict,test_correct = sess.run([accuracy,prediction,correct_prediction], feed_dict={X: test_x, Y: test_y})
        
        print("\n============Results============")
        print("Model Prediction =", train_acc)
        print("Test Prediction =", test_acc)
        
        return train_acc,test_acc, loss_ans
    
ann_mlp_train_acc, ann_mlp_test_acc , loss_ans = ann_mlp()

In [ ]:
sns.pointplot([i for i in range(len(loss_ans))], loss_ans)